# How does Julia's performance compare to C and Python?

Let's look at and benchmark the sum function:

$$\mathrm{sum}(x) = \sum_{i=1}^n x_i$$

In [1]:
x = rand(10^7);

In [2]:
sum(x)

5.000621251218744e6

In [3]:
d = Dict() # to store the measurement results

Dict{Any, Any}()

# Python

In [4]:
using BenchmarkTools
using PyCall

## numpy

In [5]:
np = pyimport("numpy")

PyObject <module 'numpy' from '/Users/crstnbr/opt/anaconda3/lib/python3.7/site-packages/numpy/__init__.py'>

In [6]:
numpy_sum = np.sum

PyObject <function sum at 0x7f910c0935f0>

In [7]:
b = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     4.984 ms (0.00% GC)
  median time:      5.306 ms (0.00% GC)
  mean time:        5.408 ms (0.00% GC)
  maximum time:     7.496 ms (0.00% GC)
  --------------
  samples:          922
  evals/sample:     1

In [8]:
d["Python (numpy)"] = minimum(b.times) / 1e6

4.98354

## hand-written

In [9]:
py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

In [10]:
mysum_py = py"mysum"

PyObject <function mysum at 0x7f910be7ee60>

In [11]:
# call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):
xpy_list = PyCall.array2py(x);

In [12]:
b = @benchmark $mysum_py($xpy_list)

BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  2
  --------------
  minimum time:     319.076 ms (0.00% GC)
  median time:      323.595 ms (0.00% GC)
  mean time:        324.182 ms (0.00% GC)
  maximum time:     334.282 ms (0.00% GC)
  --------------
  samples:          16
  evals/sample:     1

In [13]:
d["Python (hand-written)"] = minimum(b.times) / 1e6

319.076436

## built-in

In [14]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [15]:
b = @benchmark $pysum($xpy_list)

BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  2
  --------------
  minimum time:     69.473 ms (0.00% GC)
  median time:      71.635 ms (0.00% GC)
  mean time:        72.595 ms (0.00% GC)
  maximum time:     83.988 ms (0.00% GC)
  --------------
  samples:          69
  evals/sample:     1

In [16]:
d["Python (built-in)"] = minimum(b.times) / 1e6

69.472999

# C

## hand-written

In [17]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [18]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
const Clib = tempname()
using Libdl

In [19]:
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

In [20]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum (generic function with 1 method)

In [21]:
c_sum(x) ≈ sum(x)

true

In [22]:
b = @benchmark c_sum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     15.602 ms (0.00% GC)
  median time:      15.866 ms (0.00% GC)
  mean time:        16.145 ms (0.00% GC)
  maximum time:     22.042 ms (0.00% GC)
  --------------
  samples:          310
  evals/sample:     1

In [23]:
d["C"] = minimum(b.times) / 1e6

15.601941

## hand-written (with `-fast-math`)

In [24]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = @ccall Clib_fastmath.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum_fastmath (generic function with 1 method)

In [25]:
b = @benchmark c_sum_fastmath($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.689 ms (0.00% GC)
  median time:      4.939 ms (0.00% GC)
  mean time:        4.989 ms (0.00% GC)
  maximum time:     8.415 ms (0.00% GC)
  --------------
  samples:          999
  evals/sample:     1

In [26]:
d["C (fastmath)"] = minimum(b.times) / 1e6

4.68855

# Julia

## built-in

In [27]:
b = @benchmark sum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.555 ms (0.00% GC)
  median time:      4.971 ms (0.00% GC)
  mean time:        5.109 ms (0.00% GC)
  maximum time:     8.437 ms (0.00% GC)
  --------------
  samples:          976
  evals/sample:     1

In [28]:
d["Julia (built-in)"] = minimum(b.times) / 1e6

4.555254

## built-in (with `Vector{Any}`)

In [29]:
x_any = Vector{Any}(x)
b = @benchmark sum($x_any)

BenchmarkTools.Trial: 
  memory estimate:  152.59 MiB
  allocs estimate:  9999999
  --------------
  minimum time:     279.759 ms (0.00% GC)
  median time:      307.720 ms (8.74% GC)
  mean time:        305.233 ms (7.81% GC)
  maximum time:     312.379 ms (8.67% GC)
  --------------
  samples:          17
  evals/sample:     1

In [30]:
d["Julia (built-in, Any)"] = minimum(b.times) / 1e6

279.758764

## hand-written

In [31]:
function mysum(A)
    s = zero(eltype(A)) # the correct type of zero for A
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [32]:
b = @benchmark mysum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     15.823 ms (0.00% GC)
  median time:      16.022 ms (0.00% GC)
  mean time:        16.056 ms (0.00% GC)
  maximum time:     16.905 ms (0.00% GC)
  --------------
  samples:          311
  evals/sample:     1

In [33]:
d["Julia (hand-written)"] = minimum(b.times) / 1e6

15.822867

## hand-written (with `@simd`)

In [34]:
function mysum_simd(A)
    s = zero(eltype(A)) # the correct type of zero for A
    @simd for a in A
        s += a
    end
    return s
end

mysum_simd (generic function with 1 method)

In [35]:
b = @benchmark mysum_simd($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.584 ms (0.00% GC)
  median time:      4.983 ms (0.00% GC)
  mean time:        5.018 ms (0.00% GC)
  maximum time:     6.534 ms (0.00% GC)
  --------------
  samples:          993
  evals/sample:     1

In [36]:
d["Julia (hand-written, simd)"] = minimum(b.times) / 1e6

4.584162

## Summary

In [37]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 30, "."), lpad(round(value, digits=2), 10, "."))
end

Julia (built-in)....................4.56
Julia (hand-written, simd)..........4.58
C (fastmath)........................4.69
Python (numpy)......................4.98
C...................................15.6
Julia (hand-written)...............15.82
Python (built-in)..................69.47
Julia (built-in, Any).............279.76
Python (hand-written).............319.08


And of course, our hand-written Julia implementation is type-generic!

# Supplement: What about other functions?

### Log

In [38]:
@which log(1.2)

log(x::Float64) in Base.Math at special/log.jl:253

In [39]:
using BenchmarkTools

# uses the system C library
clog(x) = ccall(:log, Float64, (Float64,), x)
# uses LLVM's log
llvmlog(x) =  ccall(Symbol("llvm.log.f64"), llvmcall, Float64, (Float64,), x)

@btime log($(Ref(1.2))[])    
@btime clog($(Ref(1.2))[])    
@btime llvmlog($(Ref(1.2))[]);

  8.264 ns (0 allocations: 0 bytes)
  7.374 ns (0 allocations: 0 bytes)
  6.633 ns (0 allocations: 0 bytes)


On my Windows 10 machine:

```
  6.199 ns (0 allocations: 0 bytes)
  9.408 ns (0 allocations: 0 bytes)
  8.599 ns (0 allocations: 0 bytes)
```

### Exp

In [40]:
@which exp(1.2)

exp(x::T) where T<:Float64 in Base.Math at special/exp.jl:206

In [41]:
using BenchmarkTools

# uses the system C library
cexp(x) = ccall(:exp, Float64, (Float64,), x)
# uses LLVM's
llvmexp(x) =  ccall(Symbol("llvm.exp.f64"), llvmcall, Float64, (Float64,), x)

@btime exp($(Ref(1.2))[])    
@btime cexp($(Ref(1.2))[])    
@btime llvmexp($(Ref(1.2))[]);

  9.675 ns (0 allocations: 0 bytes)
  7.581 ns (0 allocations: 0 bytes)
  6.704 ns (0 allocations: 0 bytes)


On my Windows 10 machine:

```
  7.899 ns (0 allocations: 0 bytes)
  22.870 ns (0 allocations: 0 bytes)
  22.868 ns (0 allocations: 0 bytes)
```

### Matrix multiplication

In [42]:
N = 10
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [43]:
using LinearAlgebra

mul!(C, A, B); # "built-in", calls underlying BLAS/LAPACK

In [44]:
C ≈ A * B

true

In [45]:
using BenchmarkTools

function mul_naive!(𝐂, 𝐀, 𝐁)
   @inbounds for m ∈ axes(𝐀,1), n ∈ axes(𝐁,2)
       𝐂mn = zero(eltype(𝐂))
       for k ∈ axes(𝐀,2)
           𝐂mn += 𝐀[m,k] * 𝐁[k,n]
       end
       𝐂[m,n] = 𝐂mn
   end
end

mul_naive! (generic function with 1 method)

In [46]:
mul_naive!(C,A,B)
C ≈ A * B

true

[LoopVectorization.jl](https://github.com/chriselrod/LoopVectorization.jl)

In [47]:
using LoopVectorization

function mul_avx!(𝐂, 𝐀, 𝐁)
   @avx for m ∈ axes(𝐀,1), n ∈ axes(𝐁,2)
       𝐂mn = zero(eltype(𝐂))
       for k ∈ axes(𝐀,2)
           𝐂mn += 𝐀[m,k] * 𝐁[k,n]
       end
       𝐂[m,n] = 𝐂mn
   end
end

mul_avx! (generic function with 1 method)

In [48]:
mul_avx!(C,A,B)
C ≈ A * B

true

In [49]:
c_code = """
#include <stddef.h>
#include <math.h>

void gemm(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long n = 0; n < N; n++){
    for (long k = 0; k < K; k++){
      for (long m = 0; m < M; m++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
""";

In [50]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
const Clib_gemm = tempname()
using Libdl

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib_gemm * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

c_gemm(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = @ccall Clib_gemm.gemm(C::Ptr{Float64}, A::Ptr{Float64}, B::Ptr{Float64}, size(A,1)::Clong, size(A,2)::Clong, size(B,2)::Clong)::Cvoid

c_gemm (generic function with 1 method)

In [51]:
c_gemm(C, A, B)
C ≈ A*B

true

In [52]:
@btime mul!($C, $A, $B);
@btime mul_naive!($C,$A,$B);
@btime mul_avx!($C,$A,$B);
@btime c_gemm($C, $A, $B)

  433.005 ns (0 allocations: 0 bytes)
  930.968 ns (0 allocations: 0 bytes)
  107.735 ns (0 allocations: 0 bytes)
  667.867 ns (0 allocations: 0 bytes)


**Note for larger `N`:** BLAS is multithreaded for larger `N`. In this case our `mul_avx!` is slower than `mul!`.

In [53]:
N = 100
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [54]:
function mul_avx_threaded!(𝐂, 𝐀, 𝐁)
   @avxt for m ∈ axes(𝐀,1), n ∈ axes(𝐁,2)
       𝐂mn = zero(eltype(𝐂))
       for k ∈ axes(𝐀,2)
           𝐂mn += 𝐀[m,k] * 𝐁[k,n]
       end
       𝐂[m,n] = 𝐂mn
   end
end

mul_avx_threaded! (generic function with 1 method)

In [55]:
@btime mul_avx!($C, $A, $B);
@btime mul_avx_threaded!($C, $A, $B);
@btime mul!($C, $A, $B);

  54.136 μs (0 allocations: 0 bytes)
  10.092 μs (0 allocations: 0 bytes)
  24.109 μs (0 allocations: 0 bytes)


**Resources/more info:**

https://github.com/mitmath/18S096/blob/master/lectures/lecture1/Performance-variation.ipynb

https://github.com/mitmath/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb


**More comprehensive benchmarks:** https://chriselrod.github.io/LoopVectorization.jl/stable/examples/matrix_multiplication/#Matrix-Multiplication